In [167]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [168]:
# read in all the datasets
def read_clean(filename, date_vars = ["date"]):
    return pd.read_csv("../Data/" + filename, dtype={'fips': str}, parse_dates = date_vars)

cases = read_clean("NYT_clean.csv")
acs = read_clean("ACS Data.csv", None)
hospitals = read_clean("Hospitals_clean.csv").drop(columns=["collection_week"])
masks = read_clean("Masks_clean.csv")
mobility = read_clean("Mobility Data.csv")
IL_vaccine = read_clean("il_vaccine_clean.csv", ["Report_Date"]).drop(columns=["Unnamed: 0"]).drop_duplicates()
MO_vaccine = read_clean("MO_vaccine_clean.csv")
OH_vaccine = read_clean("oh_vaccine_clean.csv").drop(columns=["Unnamed: 0"])

In [169]:
# merge together all non-vaccine data
merged = cases.merge(acs.drop(columns=["county", "state"]), on=["fips"], how="left")\
              .merge(hospitals.drop(columns=["state"]), on=["fips", "date"], how="left")\
              .merge(masks.drop(columns=["county", "state"]), on=["fips", "date"], how="left")\
              .merge(mobility.drop(columns=["country", "state", "county"]), on=["fips", "date"], how="left")

In [170]:
# standardize vaccine data
IL_vaccine = IL_vaccine.rename(columns={"AdministeredCount": "cumulative doses",
                                        "Report_Date": "date"})
IL_vaccine['new doses'] = IL_vaccine.groupby(["state", "CountyName", "fips"])['cumulative doses'].diff()


OH_vaccine['new doses'] = OH_vaccine["vaccines_started"] + OH_vaccine["vaccines_completed"]
OH_vaccine['new doses'] = OH_vaccine["vaccines_started"] + OH_vaccine["vaccines_completed"]
OH_vaccine["cumulative doses"] = OH_vaccine.sort_values(by=["state", "county", "fips", "date"])\
                                                       .groupby(['state', "county", "fips"])['new doses']\
                                                       .cumsum(axis=0)


# stack vaccine data
vaccines = IL_vaccine[["fips", "date", "new doses", "cumulative doses"]]\
            .append(OH_vaccine[["fips", "date", "new doses", "cumulative doses"]])\
            .append(MO_vaccine[["fips", "date", "new doses", "cumulative doses"]])


# merge onto rest of the data
merged = merged.merge(vaccines, on=["fips", "date"], how="left")

merged["cumulative doses"] = merged.groupby(["fips"])['cumulative doses'].transform(lambda x: x.ffill())

merged[["new doses", "cumulative doses"]] = merged[["new doses", "cumulative doses"]].fillna(0)

In [172]:
# look at a subset that should have all the data valued basically
merged[merged["date"] > '2020-12-21'].head()

,state,fips,county,date,cumulative_cases,cumulative_deaths,new_cases,new_deaths,new_cases_7avg,new_deaths_7avg,2weeksago_cases_7avg,2weeksago_deaths_7avg,total_pop,male,perc_male,female,perc_female,age_15_19,p_age_15_19,age_20_24,p_age_20_24,age_25_34,p_age_25_34,age_35_44,p_age_35_44,age_45_54,p_age_45_54,age_55_59,p_age_55_59,age_60_64,p_age_60_64,age_median,age_62over,p_age_62over,age_65over,p_age_65over,white,p_white,black,p_black,native,p_native,asian,p_asian,hawaiian,p_hawaiian,other_race,p_other_race,hispanic,p_hispanic,housing_units,below_50_pov,below_125_pov,below_150_pov,below_185_pov,below_200_pov,below_300_pov,below_400_pov,below_500_pov,below_pov,male_below_pov,female_below_pov,age_under14,p_under14,non_white,p_non_white,total_adult_hospitalizations,total_pediatric_hospitalizations,prev_day_adult_admit_7daysum,prev_day_adult_admit_18-19_7daysum,prev_day_adult_admit_20-29_7daysum,prev_day_adult_admit_30-39_7daysum,prev_day_adult_admit_40-49_7daysum,prev_day_adult_admit_50-59_7daysum,prev_day_adult_admit_60-69_7daysum,prev_day_adult_admit_70-79_7daysum,prev_day_adult_admit_80+_7daysum,prev_day_adult_admit_unknown_7daysum,mask_mandate,retail_rec,grocery_pharm,parks,transit,workplace,residential,new doses,cumulative doses
277,IL,17001,Adams,2020-12-22,6040.0,72.0,118.0,3.0,79.0,1.0,71.0,1.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,413.0,0.0,68.0,0.0,4.0,3.0,4.0,6.0,12.0,18.0,21.0,3.0,1.0,5.0,16.0,NaN,-32.0,-28.0,7.0,0.0,9.0
278,IL,17001,Adams,2020-12-23,6051.0,72.0,11.0,0.0,60.0,1.0,64.0,1.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,413.0,0.0,68.0,0.0,4.0,3.0,4.0,6.0,12.0,18.0,21.0,3.0,1.0,20.0,37.0,NaN,-34.0,-30.0,7.0,536.0,545.0
279,IL,17001,Adams,2020-12-24,6162.0,73.0,111.0,1.0,59.0,1.0,52.0,1.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,413.0,0.0,68.0,0.0,4.0,3.0,4.0,6.0,12.0,18.0,21.0,3.0,1.0,-18.0,33.0,NaN,-44.0,-56.0,15.0,244.0,789.0
280,IL,17001,Adams,2020-12-25,6162.0,74.0,0.0,1.0,59.0,1.0,57.0,1.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,424.0,0.0,61.0,0.0,4.0,3.0,3.0,6.0,12.0,14.0,22.0,0.0,1.0,-89.0,-75.0,NaN,-54.0,-83.0,24.0,0.0,789.0
281,IL,17001,Adams,2020-12-26,6117.0,77.0,19.5,3.0,53.0,1.0,65.0,1.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,424.0,0.0,61.0,0.0,4.0,3.0,3.0,6.0,12.0,14.0,22.0,0.0,1.0,-32.0,-13.0,NaN,-54.0,-19.0,7.0,0.0,789.0


In [173]:
###############################
# Treat NAs for hospital data #
###############################

# filling na's with 0 for data points after 7-31-2020 because that's when the data should be complete
# most counties that had missing values just didn't have any hospitals in the dataset associated with it, and
# I spot-checked a few and it seemed to be counties without large hospitals.  Therefore, let's fill those in with 0's
# Also manually checked the two counties that had sporadic missings, but the surrounding dates (around the missing 
# dates) were all basically 0 so I feel okay filling in those with 0s
date_mask = (merged["date"]>='7-31-2020')
numerical_hosp_variables = hospitals.columns.tolist()[3:]
merged.loc[date_mask, numerical_hosp_variables] = merged.loc[date_mask, numerical_hosp_variables].fillna(0)

merged[date_mask].describe(datetime_is_numeric=True)

,date,cumulative_cases,cumulative_deaths,new_cases,new_deaths,new_cases_7avg,new_deaths_7avg,2weeksago_cases_7avg,2weeksago_deaths_7avg,total_pop,male,perc_male,female,perc_female,age_15_19,p_age_15_19,age_20_24,p_age_20_24,age_25_34,p_age_25_34,age_35_44,p_age_35_44,age_45_54,p_age_45_54,age_55_59,p_age_55_59,age_60_64,p_age_60_64,age_median,age_62over,p_age_62over,age_65over,p_age_65over,white,p_white,black,p_black,native,p_native,asian,p_asian,hawaiian,p_hawaiian,other_race,p_other_race,hispanic,p_hispanic,housing_units,below_50_pov,below_125_pov,below_150_pov,below_185_pov,below_200_pov,below_300_pov,below_400_pov,below_500_pov,below_pov,male_below_pov,female_below_pov,age_under14,p_under14,non_white,p_non_white,total_adult_hospitalizations,total_pediatric_hospitalizations,prev_day_adult_admit_7daysum,prev_day_adult_admit_18-19_7daysum,prev_day_adult_admit_20-29_7daysum,prev_day_adult_admit_30-39_7daysum,prev_day_adult_admit_40-49_7daysum,prev_day_adult_admit_50-59_7daysum,prev_day_adult_admit_60-69_7daysum,prev_day_adult_admit_70-79_7daysum,prev_day_adult_admit_80+_7daysum,prev_day_adult_admit_unknown_7daysum,mask_mandate,retail_rec,grocery_pharm,parks,transit,workplace,residential,new doses,cumulative doses
count,117111,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,1.171110e+05,1.171110e+05,117111.000000,1.171110e+05,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,1.171110e+05,117111.000000,1.171110e+05,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,1.171110e+05,117111.000000,1.171110e+05,117111.000000,117111.000000,1.171110e+05,1.171110e+05,1.171110e+05,1.171110e+05,1.171110e+05,1.171110e+05,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,1.171110e+05,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,117111.000000,91371.000000,63414.000000,56228.000000,17927.000000,33013.000000,97551.000000,65760.000000,117111.000000,1.171110e+05
mean,2020-12-28 04:33:37.409124608,5968.440522,117.095973,31.349601,0.506754,31.407528,0.450812,31.468410,0.444698,1.052472e+05,5.167189e+04,49.992793,5.357533e+04,50.007721,6918.520839,6.417573,7123.782514,6.134980,13987.749648,11.457057,12842.195464,11.418598,13698.454159,12.859055,7360.588228,7.450544,6878.258994,7.170483,42.185661,21011.070198,23.483985,17023.871114,19.272420,8.162746e+04,91.122325,1.394614e+04,4.270402,357.139628,0.581623,3592.966263,1.003852,53.979157,0.050931,2764.147945,0.801762,8.822256e+03,3.606252,4.666573e+04,6256.414402,18246.184201,2.247946e+04,2.876334e+04,3.143410e+04,4.840526e+04,6.309701e+04,7.447668e+04,13932.418039,6185.031859,7747.386181,19413.801957,17.829321,2.361976e+04,8.877675,122.765317,1.548992,16.773950,0.185585,1.071693,1.399057,1.853302,2.997703,3.899864,4.107735,3.753123,0.542759,0.699445,-4.253335,1.480792,33.898589,-5.662375,-18.568554,5.951551,186.511950,1.260754e+04
min,2020-07-31 00:00:00,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.027000e+03,9.760000e+02,44.600000,1.022000e+03,36.700000,77.000000,2.900000,84.000000,2.300000,158.000000,5.400000,197.000000,7.400000,210.000000,8.300000,174.000000,4.000000,159.000000,4.100000,27.600000,612.000000,11.200000,500.000000,8.300000,1.982000e+03,46.500000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.276000e+03,108.000000,304.000000,4.300000e+02,5.990000e+02,6.580000e+02,9.500000e+02,1.365000e+03,1.673000e+03,223.000000,86.000000,137.000000,247.000000,9.400000,3.600000e+01,0.700000,0.000000,0.0

In [174]:
###############################
# Treat NAs for mask data #
###############################

# since mask mandate data ends 3-22-2021 and our covid data goes on until 5-27-2021, let's forward
# fill NA's after 3-22-2021 (I've hand checked and the mask mandates have not changed for our three states)
mask_date_mask = (merged["date"] >= '3-22-2021')
merged.loc[mask_date_mask, 'mask_mandate'] = merged.loc[mask_date_mask, 'mask_mandate'].fillna(method='ffill', 
                                                                                               limit = 20)
merged.loc[mask_date_mask & merged["mask_mandate"].isnull()].head()

,state,fips,county,date,cumulative_cases,cumulative_deaths,new_cases,new_deaths,new_cases_7avg,new_deaths_7avg,2weeksago_cases_7avg,2weeksago_deaths_7avg,total_pop,male,perc_male,female,perc_female,age_15_19,p_age_15_19,age_20_24,p_age_20_24,age_25_34,p_age_25_34,age_35_44,p_age_35_44,age_45_54,p_age_45_54,age_55_59,p_age_55_59,age_60_64,p_age_60_64,age_median,age_62over,p_age_62over,age_65over,p_age_65over,white,p_white,black,p_black,native,p_native,asian,p_asian,hawaiian,p_hawaiian,other_race,p_other_race,hispanic,p_hispanic,housing_units,below_50_pov,below_125_pov,below_150_pov,below_185_pov,below_200_pov,below_300_pov,below_400_pov,below_500_pov,below_pov,male_below_pov,female_below_pov,age_under14,p_under14,non_white,p_non_white,total_adult_hospitalizations,total_pediatric_hospitalizations,prev_day_adult_admit_7daysum,prev_day_adult_admit_18-19_7daysum,prev_day_adult_admit_20-29_7daysum,prev_day_adult_admit_30-39_7daysum,prev_day_adult_admit_40-49_7daysum,prev_day_adult_admit_50-59_7daysum,prev_day_adult_admit_60-69_7daysum,prev_day_adult_admit_70-79_7daysum,prev_day_adult_admit_80+_7daysum,prev_day_adult_admit_unknown_7daysum,mask_mandate,retail_rec,grocery_pharm,parks,transit,workplace,residential,new doses,cumulative doses
388,IL,17001,Adams,2021-04-12,8131.0,145.0,3.0,0.0,12.0,0.0,4.0,0.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,64.0,3.0,9.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,0.0,NaN,-3.0,9.0,NaN,-34.0,-16.0,3.0,27.0,41394.0
389,IL,17001,Adams,2021-04-13,8132.0,145.0,1.0,0.0,10.0,0.0,4.0,0.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,64.0,3.0,9.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,0.0,NaN,3.0,3.0,NaN,NaN,-17.0,3.0,18.0,41412.0
390,IL,17001,Adams,2021-04-14,8149.0,145.0,17.0,0.0,9.0,0.0,3.0,0.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,64.0,3.0,9.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,0.0,NaN,2.0,7.0,NaN,-11.0,-16.0,2.0,133.0,41545.0
391,IL,17001,Adams,2021-04-15,8168.0,146.0,19.0,1.0,9.0,0.0,5.0,0.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,64.0,3.0,9.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,0.0,NaN,-5.0,11.0,NaN,-33.0,-18.0,3.0,89.0,41634.0
392,IL,17001,Adams,2021-04-16,8200.0,146.0,32.0,0.0,12.0,0.0,5.0,0.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,77.0,5.0,11.0,0.0,0.0,3.0,0.0,3.0,3.0,4.0,3.0,0.0,NaN,-4.0,-3.0,NaN,-3.0,-13.0,1.0,79.0,41713.0


In [175]:
# drop MI data for now since we don't have vaccination data
merged = merged[merged["state"]!="MI"]

In [176]:
# create cases_next_week variable that's the cases 7 days from now
merged["cases_next_week"]= merged.groupby(["state", "fips", "county"])['new_cases_7avg']\
                                            .shift(-7)

merged[["state", "fips", "county", "new_cases_7avg", "cases_next_week", "date"]].head(50)

,state,fips,county,new_cases_7avg,cases_next_week,date
0,IL,17001,Adams,1.0,0.0,2020-03-20
1,IL,17001,Adams,0.0,0.0,2020-03-21
2,IL,17001,Adams,0.0,0.0,2020-03-22
3,IL,17001,Adams,0.0,0.0,2020-03-23
4,IL,17001,Adams,0.0,0.0,2020-03-24
5,IL,17001,Adams,0.0,0.0,2020-03-25
6,IL,17001,Adams,0.0,0.0,2020-03-26
7,IL,17001,Adams,0.0,0.0,2020-03-27
8,IL,17001,Adams,0.0,0.0,2020-03-28
9,IL,17001,Adams,0.0,0.0,2020-03-29


In [177]:
# make sure there's no negative cases values
bool((merged.loc[:,"cumulative_cases":"2weeksago_deaths_7avg"] >= 0).all().mean())

True

In [178]:
# export dataset
merged.to_csv("../Data/merged.csv", index=False)